Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [436]:
from random import choices, random, randint
import numpy as np
from copy import deepcopy

import lab9_lib

# GOAL: maximize fitness, minimize calls

In [437]:
fitness = lab9_lib.make_problem(5)
for n in range(20):
    ind = choices([0, 1], k=100)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

0001101101100100111001001001100111100100000100101101010001100011000010001111011101100101000011110010: 13.71%
1011111111101010111110110010111100000100111111001111101001101010101110110110000011101100001010000010: 24.78%
1101100001111110111000110111000110101111010010100101011100000011110110101100001111111110100101000010: 12.68%
1001000111101001111110010011100001010000111101101001111111011111110100100001001000110110110000100111: 12.58%
1001111110101011000101010100011011001111110010000011110001000000111000101001010011110011011011100111: 11.69%
0110001010111110010011111111100011000001110001000000101010001001100110011001111011000010111011110011: 31.92%
0101110100101100110001110011110010110000110000100110000111010011001111010010110000001011010100101011: 11.78%
0000110111010011110100000010000111010100000111111001000110000101101011101010011001101101000011101010: 9.89%
0000001001010110010100110101010010011110011010100000000110000100001001110100111011101110010110100011: 11.90%
11100110000110101011

Our code below

In [438]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 50
LOCI = 1000
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = 0.25
BIT_FLIP_PROBABILITY = 0.15
NUM_GENERATION = 200


class Individual:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float("-inf")

In [439]:
# Mutation / recombination or both
def parent_selection(population: list[Individual]) -> list[Individual]:
    # we also want to take the last best one.
    parents_idx = np.random.choice(
        range(len(population)), size=TOURNAMENT_SIZE - 1, replace=False
    )
    parents = [population[idx] for idx in parents_idx if idx != 0]
    return [population[0], max(parents, key=lambda i: i.fitness)]


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI - 1)
    new_ind = Individual()
    new_ind.genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    assert len(new_ind.genotype) == LOCI
    return new_ind


# This xover function returns a child whose genome
# is created proportionally to the fitenss of parents
def uniform_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    p1 = ind1.fitness / (ind1.fitness + ind2.fitness)
    gene = [
        np.random.choice([ind1.genotype[i], ind2.genotype[i]], p=[p1, 1 - p1])
        for i in range(LOCI)
    ]
    new_ind = Individual()
    new_ind.genotype = gene
    return new_ind


def mutate(parent: Individual) -> Individual:
    new_offspring = deepcopy(parent)
    for i in range(LOCI):
        if random() < BIT_FLIP_PROBABILITY:
            new_offspring.genotype[i] = int(not new_offspring.genotype[i])
    return new_offspring


def offspring_generation(
    parent1: Individual, parent2: Individual
) -> Individual:
    if random() < MUTATION_PROBABILITY:
        # mutation
        return mutate(parent1)
    else:
        # cross_over
        return uniform_cut_xover(parent1, parent2)


def ea() -> Individual:
    # starting pouplation of POPULATION_SIZE individuals
    population = [Individual() for _ in range(POPULATION_SIZE)]
    for p in population:
        p.fitness = fitness(p.genotype)
    old_best_fitness = 0
    best_fitness = population[0].fitness
    # for _ in range(NUM_GENERATION)
    gen = 0
    while best_fitness != old_best_fitness or gen < NUM_GENERATION:
        old_best_fitness = best_fitness
        # best_old_one = best_one
        best_parents = parent_selection(population)
        for _ in range(OFFSPRING_SIZE):
            offspring = offspring_generation(best_parents[0], best_parents[1])
            offspring.fitness = fitness(offspring.genotype)
            population.extend([offspring])

        population.sort(key=lambda i: i.fitness, reverse=True)
        # always keep the first POPULATION_SIZE best individuals
        population = population[:POPULATION_SIZE]
        best_fitness = population[0].fitness
        gen += 1
    return population[0]

In [440]:
# instance = [1, 2, 5, 10]
instance = [2]
for k in instance:
    fitness = lab9_lib.make_problem(k)
    print(f"Best individual fitness: {ea().fitness}")
    print(f"Fitness calls: {fitness.calls}")

Best individual fitness: 0.594
Fitness calls: 10050
